In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import wandb
from pytorch_lightning.loggers import WandbLogger
import torch
from Models.EGformer import EGformer
import torch.nn as nn
import torch.optim as optim
from ocpmodels.datasets import LmdbDataset
from torch.utils.data import random_split
import torch_geometric.loader as geom_loader
import torch_geometric.data as data
from typing import Any
import yaml
import torch.nn.init as init
import torch.nn as nn

c:\Users\lhuang37\.conda\envs\ocp-models\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:root:Invalid setup for SCN. Either the e3nn library or Jd.pt is missing.


In [2]:
#Save the model hyperparameters to a YAML file
model_hparams ={"num_atoms":0,
                "bond_feat_dim":0,
                "num_targets":0,
                "num_heads":4,
                "batch_size":6
                }

with open('params/model_hparams.yml', 'w') as file:
    yaml.dump(model_hparams, file)

In [6]:

with open('params/model_hparams.yml', 'r') as file:
    loaded_model_hparams = yaml.load(file, Loader=yaml.FullLoader)

# Create the model using the loaded hyperparameters
model = EGformer(**loaded_model_hparams)
checkpoint_path='params/gemnet_oc_base_oc20_oc22.pt'
pretrained_state_dict = torch.load(checkpoint_path)['state_dict']
new_model_state_dict = model.state_dict()
filtered_pretrained_state_dict = {k.strip('module.module.'): v for k, v in pretrained_state_dict.items() if k.strip('module.module.') in new_model_state_dict}
new_model_state_dict.update(filtered_pretrained_state_dict)
model.load_state_dict(new_model_state_dict)
for param_name, param in model.named_parameters():

    if param_name in filtered_pretrained_state_dict.keys():
        
        param.requires_grad = False

In [ ]:
# with open('params/model_hparams.yml', 'r') as file:
#     loaded_model_hparams = yaml.load(file, Loader=yaml.FullLoader)

# # Create the model using the loaded hyperparameters
# model = Egformer(**loaded_model_hparams)
# checkpoint_path='params/gemnet_oc_base_oc20_oc22.pt'
# pretrained_state_dict = torch.load(checkpoint_path)['state_dict']
# new_model_state_dict = model.arc.state_dict()
# filtered_pretrained_state_dict = {k.strip('module.module.'): v for k, v in pretrained_state_dict.items() if k.strip('module.module.') in new_model_state_dict}
# new_model_state_dict.update(filtered_pretrained_state_dict)
# model.arc.load_state_dict(new_model_state_dict)
# for param_name, param in model.named_parameters():

#     if param_name.replace('arc.','') in filtered_pretrained_state_dict.keys():
        
#         param.requires_grad = False

In [7]:
f_paras,t_paras=0,0
for param_name,param in model.named_parameters():
    if param.requires_grad is False:
        f_paras+=1
    else:
        t_paras+=1
print('Freeze params is',f_paras)
print('Need optimiz params is',t_paras)

Freeze params is 238
Need optimiz params is 131


In [9]:
warmup_epochs=2
decay_epochs=3
    
y_mean=-7
y_std=6
num_epochs=10
batch_size = 6
learning_rate=0.001
CHECKPOINT_PATH="./checkpoints"
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset=LmdbDataset({"src":"/shareddata/ocp/ocp22/oc22_trajectories/trajectories/Transformer_clean_valid/data.0000.lmdb"})
model= nn.DataParallel(model) 
model.to(DEVICE)
criterion=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
warmup_scheduler = optim.lr_scheduler.LambdaLR(optimizer,lr_lambda=lambda epoch: (epoch+1)/warmup_epochs)
decay_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=decay_epochs,gamma=0.1)

In [ ]:
train_length = int(0.8 * len(dataset))
val_length = len(dataset) - train_length

# Split the dataset into train and validation
train_dataset, val_dataset =random_split(dataset, [train_length, val_length])
train_loader = geom_loader.DataLoader(train_dataset, batch_size=6,drop_last=True)
val_loader = geom_loader.DataLoader(val_dataset, batch_size=6,drop_last=True)

In [ ]:
data = next(iter(train_loader))
batched_data = data.batch
print(batched_data.shape)

In [ ]:
from tqdm import tqdm

In [10]:
def get_loss(predictions, targets, y_mean=-7, y_std=6):
    masks = (targets/ targets.natoms - y_mean) / y_std
    mask_loss = nn.MSELoss()
    loss = mask_loss(predictions.view(-1, 1), masks.view(-1, 1))
    accuracy = torch.abs(predictions.view(-1, 1) - masks.view(-1, 1))
    return loss, accuracy

In [11]:
def train_fn(data_loader, model, optimizer, device, optimize_after=8):
    model.train()
    total_loss = 0.0
    iteration = 0

    for images in tqdm(data_loader):
        images = images.to(device)
        optimizer.zero_grad()
        predictions = model(images)
        targets = images.y
        loss, acc = get_loss(predictions, targets)
        loss.backward()

        # Accumulate gradients for a specified number of iterations
        iteration += 1
        if iteration % optimize_after == 0:
            optimizer.step()
            iteration = 0
            total_loss += loss.item()

    return total_loss / (len(data_loader) // optimize_after)

In [ ]:
def eval_fn(data_loader,model):
    model.eval()
    total_loss=0.0
    total_acc=0    
    with torch.no_grad():
        for images in tqdm(data_loader):
            model=model.to(DEVICE) 
            images=images.to(DEVICE)                      
            loss,acc=model(images)
            total_loss+=loss.item()
            total_acc+=acc.item()

    return total_loss/len(data_loader),total_acc/len(data_loader)

In [ ]:
best_valid_loss=np.Inf
for epoch in range(num_epochs):
    train_loss=train_fn(train_loader,model,optimizer,device=DEVICE)
    valid_loss,acc=eval_fn(val_loader,model,device=DEVICE)
    if epoch<warmup_epochs:
        warmup_scheduler.step()
    else:
        decay_scheduler.step()    
    if valid_loss< best_valid_loss:
        torch.save(model.state_dict(),'best_model_n.pt')
        print('saved-model')
        best_valid_loss=valid_loss
    current_lr=optimizer.param_groups[0]['lr']
    
    print(f'epoch:{epoch+1} Train_loss:{train_loss} Valid_loss:{valid_loss} Valid acc:{acc} lr:{current_lr}')